# Projeto ML

Trabalho de:

* Bárbara Simões Neto up
* Beatriz Castro up
* Rodrigo Couto up202104696


sites usados até então:

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html

https://link.springer.com/article/10.1007/s10618-021-00737-9 (so vi as imagens para perceber o que era os diferentes tipos de datasets que o chatzaome deu, elas sao bastantes intuitivas)




In [8]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_moons, make_circles, make_blobs
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC  



In [ ]:

#Gerar datasets

def generate_mixed_dataset(n_samples, n_features, n_categorical, n_ordinal, n_integer, 
                           n_classes, class_balance=None, noise=0.0, 
                           dataset_type='linear', random_state=42):

    np.random.seed(random_state)

    if dataset_type == 'linear':
        X, y = make_classification(n_samples=n_samples, n_features=n_features, 
                                  n_informative=n_features, n_redundant=0, 
                                  n_classes=n_classes, weights=class_balance, 
                                  flip_y=noise, random_state=random_state, class_sep=2)
    elif dataset_type == 'moons':
        X, y = make_moons(n_samples=n_samples, noise=noise, random_state=random_state)
        if n_features > 2:
            extra_features = np.random.rand(n_samples, n_features - 2) * 100
            X = np.hstack([X, extra_features])
    elif dataset_type == 'circles':
        X, y = make_circles(n_samples=n_samples, noise=noise, random_state=random_state)
        if n_features > 2:
            extra_features = np.random.rand(n_samples, n_features - 2) * 100
            X = np.hstack([X, extra_features])
    elif dataset_type == 'blobs':
        X, y = make_blobs(n_samples=n_samples, centers=n_classes, cluster_std=noise*10, 
                          random_state=random_state)
        if n_features > 2:
            extra_features = np.random.rand(n_samples, n_features - 2) * 100
            X = np.hstack([X, extra_features])
    else:
        raise ValueError("Tipo de dataset não suportado. Use 'linear', 'moons', 'circles' ou 'blobs'.")

    categorical_data = np.random.choice(['A', 'B', 'C', 'D', 'E'], size=(n_samples, n_categorical))

    ordinal_data = np.random.randint(1, 6, size=(n_samples, n_ordinal))  #podem mudar estes valores para criar dataset mais especificos

    integer_data = np.random.randint(0, 100, size=(n_samples, n_integer))  #podem mudar estes valores para criar dataset mais especificos

    # Combina todos os dados em um único array
    data = np.hstack([X, categorical_data, ordinal_data, integer_data])

    columns = [f'Continuous_{i+1}' for i in range(n_features)] + \
              [f'Categorical_{i+1}' for i in range(n_categorical)] + \
              [f'Ordinal_{i+1}' for i in range(n_ordinal)] + \
              [f'Integer_{i+1}' for i in range(n_integer)]
    df = pd.DataFrame(data, columns=columns)

    # Adiciona a coluna target
    df['Target'] = y

    # Converte as colunas para os tipos corretos
    for col in df.columns:
        if col.startswith('Continuous'):
            df[col] = df[col].astype(float)
        elif col.startswith('Ordinal') or col.startswith('Integer'):
            df[col] = df[col].astype(int)

    return df

# SVM linear

Linear SVMs work best with specific types of datasets. They are ideal for linearly separable data, where classes can be clearly divided by a straight line. They also perform well with high-dimensional data, like text or datasets with many features, as they focus on finding a linear decision boundary. Numerical data (continuous or integer values) works best, while categorical data needs to be transformed (e.g., using one-hot encoding). However, linear SVMs have limitations. They struggle with large datasets due to high computational costs and memory usage. They also perform poorly on imbalanced datasets, often favoring the majority class, and on noisy data where classes overlap significantly. For multiclass problems, additional techniques like One-vs-Rest or One-vs-One are needed, as linear SVMs are designed for binary classification. In summary, linear SVMs are best suited for small to medium-sized, high-dimensional, linearly separable datasets with balanced classes and minimal noise.

In [34]:
#gerar dataset adequado

df_svm_linear = generate_mixed_dataset(n_samples=500, n_features=20, n_categorical=0, n_ordinal=0, n_integer=8,
                                                  n_classes=2, class_balance=None, noise=0.1, dataset_type='linear')

print("Best Dataset for Linear SVM:")
print(df_svm_linear.head())


Best Dataset for Linear SVM:
   Continuous_1  Continuous_2  Continuous_3  Continuous_4  Continuous_5  \
0     -2.307608      1.668358      2.252106     -1.968564     -4.929186   
1      4.389963     -0.578385     -2.722650     -2.982947      7.632946   
2     -1.897597      2.994106      0.755542     -5.697095      2.729707   
3      3.867551     -1.917197      2.019131     -0.850470      2.443705   
4     -2.549646      2.202160      2.300397      0.795324     -3.497897   

   Continuous_6  Continuous_7  Continuous_8  Continuous_9  Continuous_10  ...  \
0      0.772611     -5.188637      0.909699      0.951789       1.001922  ...   
1      3.088414     -2.569981      3.337391     -6.984058      -0.624044  ...   
2     -0.131957     -1.698201      0.369110     -0.722678      -1.358863  ...   
3      3.185501     -4.957228     -4.213375     -0.848798       1.247323  ...   
4     -2.680108     -2.706487      5.675596      2.201553      -4.804617  ...   

   Continuous_20  Integer_1  Inte

In [35]:
X = df_svm_linear.drop('Target', axis=1)  
y = df_svm_linear['Target']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standerizar as features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Training model
model = LinearSVC(C=0.1,random_state=42)  


model.fit(X_train, y_train)



#Predicts
y_pred = model.predict(X_test)

#novo dataset para avaliar o modelo
df_svm_linear_2 = generate_mixed_dataset(n_samples=150, n_features=20, n_categorical=0, n_ordinal=0, n_integer=8,
                                           n_classes=2, class_balance=None, noise=0.1, dataset_type='linear')



X_2 = df_svm_linear_2.drop('Target', axis=1)
y_2 = df_svm_linear_2['Target']


X_2 = scaler.transform(X_2)

y_pred_2 = model.predict(X_2)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# cross-validation 
cv_scores_2 = cross_val_score(model, X_2, y_2, cv=5, scoring='accuracy')

# Resultados
print(f"Cross-Validation Accuracy: {cv_scores_2.mean():.2f} (± {cv_scores_2.std():.2f})")
print("\nFold Accuracies:")
for i, score in enumerate(cv_scores_2, start=1):
    print(f"Fold {i}: {score:.2f}")

Cross-Validation Accuracy: 0.83 (± 0.04)

Fold Accuracies:
Fold 1: 0.80
Fold 2: 0.83
Fold 3: 0.83
Fold 4: 0.80
Fold 5: 0.90


# SVM RBF

The SVM RBF (Radial Basis Function) kernel is ideal for non-linear and complex datasets, as it can capture intricate patterns where classes are not linearly separable. It also handles noisy or overlapping data better than linear SVMs, creating flexible decision boundaries. However, its performance depends heavily on hyperparameter tuning, such as the gamma value. For other aspects, like handling high-dimensional data, numerical features, large datasets, imbalanced classes, and multiclass problems, it behaves similarly to linear SVMs.

In [42]:
#gerar dataset
df_svm_rbf = generate_mixed_dataset(n_samples=500, n_features=20, n_categorical=0,n_ordinal=1,n_integer=8,
    n_classes=2,class_balance=None,noise=0.2,dataset_type='moons', random_state=42)

print("\nMelhor Dataset para SVM RBF:")
print(df_svm_rbf.head())



Melhor Dataset para SVM RBF:
   Continuous_1  Continuous_2  Continuous_3  Continuous_4  Continuous_5  \
0      0.830858     -0.334342     37.454012     95.071431     73.199394   
1      0.991710      0.879000     43.194502     29.122914     61.185289   
2      1.107245     -0.470344     30.461377      9.767211     68.423303   
3     -0.140899      1.033148     59.789998     92.187424      8.849250   
4      0.405592      1.328529      0.552212     81.546143     70.685734   

   Continuous_6  Continuous_7  Continuous_8  Continuous_9  Continuous_10  ...  \
0     59.865848     15.601864     15.599452      5.808361      86.617615  ...   
1     13.949386     29.214465     36.636184     45.606998      78.517596  ...   
2     44.015249     12.203823     49.517691      3.438852      90.932040  ...   
3     19.598286      4.522729     32.533033     38.867729      27.134903  ...   
4     72.900717     77.127035      7.404465     35.846573      11.586906  ...   

   Ordinal_1  Integer_1  Integer

In [43]:
X = df_svm_rbf.drop('Target', axis=1)  
y = df_svm_rbf['Target']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standerizar as features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training model
model = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42) 

model.fit(X_train, y_train)

# Predicts
y_pred = model.predict(X_test)

# novo dataset para avaliar o modelo
df_svm_rbf_2 = generate_mixed_dataset(n_samples=200, n_features=20, n_categorical=0, n_ordinal=1, n_integer=8,
                                     n_classes=2, class_balance=None, noise=0.2, dataset_type='moons')

X_2 = df_svm_rbf_2.drop('Target', axis=1)
y_2 = df_svm_rbf_2['Target']

X_2 = scaler.transform(X_2)

y_pred_2 = model.predict(X_2)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# cross-validation 
cv_scores_2 = cross_val_score(model, X_2, y_2, cv=5, scoring='accuracy')

# Resultados
print(f"Cross-Validation Accuracy: {cv_scores_2.mean():.2f} (± {cv_scores_2.std():.2f})")
print("\nFold Accuracies:")
for i, score in enumerate(cv_scores_2, start=1):
    print(f"Fold {i}: {score:.2f}")

Cross-Validation Accuracy: 0.88 (± 0.04)

Fold Accuracies:
Fold 1: 0.90
Fold 2: 0.80
Fold 3: 0.90
Fold 4: 0.90
Fold 5: 0.88
